In [7]:
from pydrive.auth import GoogleAuth
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=849244129200-hdfqohk1rs46hjekajgu7pa4jqrn9sqj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [ ]:
# Вот полный скрипт, который автоматически формирует google_drive_links.csv, подключаясь к Google Drive, 
# забирая все ссылки из указанной папки, и сохраняет результат в CSV-файл.

In [9]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd

# === 🌐 Авторизация в Google Drive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # Открывает окно авторизации
drive = GoogleDrive(gauth)

# === 📂 ID папки на Google Drive
folder_id = '1em81MElkxnaue5r92e9hPGezgOuw4nRL'

# === 🔄 Запрос списка файлов из папки
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

# === 📋 Сохранение данных в DataFrame
data = {
    'filename': [file['title'] for file in file_list],
    'file_id': [file['id'] for file in file_list],
    'link': [f"https://drive.google.com/uc?export=view&id={file['id']}" for file in file_list]
}

df_links = pd.DataFrame(data)

# === 💾 Сохранение в CSV
output_csv = 'csv/google_drive_links.csv'
df_links.to_csv(output_csv, index=False)

print(f"✅ Ссылки успешно сохранены в {output_csv}")

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=849244129200-hdfqohk1rs46hjekajgu7pa4jqrn9sqj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
✅ Ссылки успешно сохранены в csv/google_drive_links.csv


In [ ]:
# Вот полный скрипт, который берет ссылки из google_drive_links.csv, 
# подставляет их в photos.geojson и сохраняет в новый файл photos_updated.geojson.

In [27]:
import pandas as pd
import json

# === 📁 Чтение данных
df_links = pd.read_csv('csv/google_drive_links.csv')

# === 🌐 Загрузка GeoJSON
with open('geojson/photos.geojson', 'r', encoding='utf-8') as file:
    geojson_data = json.load(file)

# === 🔄 Обновление ссылок в GeoJSON с использованием thumbnail
for feature in geojson_data['features']:
    filename = feature['properties']['filename']
    matching_row = df_links[df_links['filename'] == filename]
    if not matching_row.empty:
        file_id = matching_row.iloc[0]['file_id']
        new_link = f"https://drive.google.com/thumbnail?id={file_id}"
        feature['properties']['image'] = new_link

# === 💾 Сохранение обновленного GeoJSON
output_path = 'geojson/photos.geojson'
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(geojson_data, file, ensure_ascii=False, indent=4)

print(f"✅ Обновленный GeoJSON сохранен в {output_path}")

✅ Обновленный GeoJSON сохранен в geojson/photos.geojson
